In [14]:
#!pip install beautifulsoup4 requests selenium

In [15]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import re

Collecting Signal IDs

Viewing IDs List

In [16]:
IDs = pd.read_csv("signal_list.csv")
IDs.describe()


,Signal Id
count,2.091000e+03
mean,1.756951e+06
std,2.352849e+05
min,8.527800e+04
25%,1.733086e+06
50%,1.838356e+06
75%,1.903918e+06
max,1.943806e+06


View Signal Information

In [17]:
#signals = pd.read_csv("signal_details.csv")
#signals.describe()

In [22]:
base_url_singleID = 'https://www.mql5.com/en/signals/'
with open('signal_details.csv', 'w', newline='') as signal_detailsFile:
  writer = csv.writer(signal_detailsFile)
  writer.writerow(['Signal Id','Trades','ProfitTrades','LossTrades','BestTrades','WorstTrades','GrossProfit','GrossLoss','SharpeRatio','RecoveryFactor','ProfitFactor','ExpectedPayoff','AlgoTrading'])
#load the csv containing Signal IDs
for row in IDs.iloc[367:]['Signal Id']:
    url = base_url_singleID + str(row) + "?source=Unknown#!tab=stats"

    #Crawl each signal
    response = requests.get(url)
    print(response.status_code)
    if response.status_code == 403:
      break
    soup = BeautifulSoup(response.content, 'html.parser')
    
    values = soup.find_all('div', class_='s-data-columns__value')
    if len(values) == 0:
        continue
    # Loop over each product and extract the data
    signalTrades = values[0].text 


# Get total number of profit trades
    s = values[1].text
    number_regex = r"([\d\s]+)\s*\(.*\)"
    match = re.search(number_regex, s)
    if match:
        number = match.group(1)
        # Remove spaces and normalize the number
        normalized_number = float(number.replace(" ", "").replace(",", ""))
    signalProfitTrades = normalized_number

    # Get total number of loss trades

    s = values[2].text
    number_regex = r"([\d\s]+)\s*\(.*\)"
    match = re.search(number_regex, s)
    if match:
        number = match.group(1)
        # Remove spaces and normalize the number
        normalized_number = float(number.replace(" ", "").replace(",", ""))
    signalLossTrades = normalized_number

    signalBestTrade = values[3].find('i').text
    signalWorstTrade = values[4].find('i').text

    signalGrossProfit = values[5].find('i').text
    signalGrossLoss = values[6].find('i').text



    signalSharpeRatio = values[9].text

    signalRecoveryFactor = values[15].text


    signalProfitFactor = values[18].text

    signalExpectedPayoff = values[19].find('i').text

    # Find the <svg> element

    if "Algo trading" in values[25].parent.text:
      pattern = r'%'
      signalAlgoTrading = re.sub(pattern, '', values[25].text)
    else :
      pattern = r'%'
      signalAlgoTrading = re.sub(pattern, '', values[26].text)

    values2 = soup.find_all('div', class_='s-list-info__value')
    signalGrowth = re.sub(pattern, '', values2[0].text) #percentage
    with open('signal_details.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([row,signalTrades,signalProfitTrades,signalLossTrades,signalBestTrade,signalWorstTrade,signalGrossProfit,signalGrossLoss,signalSharpeRatio,signalRecoveryFactor,signalProfitFactor,signalExpectedPayoff,signalAlgoTrading])

200
200
200
200
200
200
200
200


ConnectTimeout: HTTPSConnectionPool(host='www.mql5.com', port=443): Max retries exceeded with url: /en/signals/1809558?source=Unknown (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000024C03E86250>, 'Connection to 200.105.215.22 timed out. (connect timeout=None)'))